In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

26367


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
26362,2021-07-25,47,沖縄県,209,22731,3,231
26363,2021-07-26,47,沖縄県,116,22847,2,233
26364,2021-07-27,47,沖縄県,354,23201,0,233
26365,2021-07-28,47,沖縄県,347,23548,0,233
26366,2021-07-29,47,沖縄県,392,23940,2,235


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,12342,10596,155
43,香川県,20757,2214,32
44,高知県,21879,2056,29
45,鳥取県,17391,711,2
46,鹿児島県,25806,3924,39


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
26360,2021-07-23,47,沖縄県,100,22423,0,224,762.0
26361,2021-07-24,47,沖縄県,99,22522,4,228,780.0
26362,2021-07-25,47,沖縄県,209,22731,3,231,919.0
26363,2021-07-26,47,沖縄県,116,22847,2,233,1000.0
26364,2021-07-27,47,沖縄県,354,23201,0,233,1200.0
26365,2021-07-28,47,沖縄県,347,23548,0,233,1378.0
26366,2021-07-29,47,沖縄県,392,23940,2,235,1617.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
26357,2021-07-20,47,沖縄県,154,22001,0,220,559.0,3.400000,0.442353,1.369231
26358,2021-07-21,47,沖縄県,169,22170,4,224,645.0,0.097403,0.416707,1.036145
26359,2021-07-22,47,沖縄県,153,22323,0,224,738.0,-0.094675,0.442769,1.550000
26360,2021-07-23,47,沖縄県,100,22423,0,224,762.0,-0.346405,0.355187,0.315789
26361,2021-07-24,47,沖縄県,99,22522,4,228,780.0,-0.010000,0.344360,0.222222
26362,2021-07-25,47,沖縄県,209,22731,3,231,919.0,1.111111,0.522491,1.985714
26363,2021-07-26,47,沖縄県,116,22847,2,233,1000.0,-0.444976,0.530351,2.314286
26364,2021-07-27,47,沖縄県,354,23201,0,233,1200.0,2.051724,0.337740,1.298701
26365,2021-07-28,47,沖縄県,347,23548,0,233,1378.0,-0.019774,0.321001,1.053254
26366,2021-07-29,47,沖縄県,392,23940,2,235,1617.0,0.129683,0.353052,1.562092


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
560,2021-07-29,北海道,260,0,1072.0,0.145374,0.146511,0.857143
1121,2021-07-29,青森県,25,0,55.0,2.125000,NaN,5.250000
1682,2021-07-29,岩手県,12,0,83.0,-0.294118,0.192066,1.400000
2243,2021-07-29,宮城県,36,0,165.0,-0.181818,0.669752,1.000000
2804,2021-07-29,秋田県,5,0,20.0,0.250000,0.464286,1.500000


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,13464,5685,114,2021-07-29,34,0,146.0,-0.081081,0.122182,0.259259
1,京都府,14586,17966,249,2021-07-29,164,0,678.0,-0.062857,0.244878,1.309859
2,佐賀県,23001,2699,24,2021-07-29,25,0,70.0,0.315789,0.536609,2.125000
3,兵庫県,15708,43394,1316,2021-07-29,280,1,1139.0,0.102362,0.372346,0.879195
4,北海道,561,43890,1424,2021-07-29,260,0,1072.0,0.145374,0.146511,0.857143


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,13464,5685,114,2021-07-29,34,0,146.0,-0.081081,0.122182,0.259259,1813859
1,京都府,14586,17966,249,2021-07-29,164,0,678.0,-0.062857,0.244878,1.309859,2545899
2,佐賀県,23001,2699,24,2021-07-29,25,0,70.0,0.315789,0.536609,2.125000,823810
3,兵庫県,15708,43394,1316,2021-07-29,280,1,1139.0,0.102362,0.372346,0.879195,5549568
4,北海道,561,43890,1424,2021-07-29,260,0,1072.0,0.145374,0.146511,0.857143,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,561,43890,1424,2021-07-29,260,0,1072.0,0.145374,0.146511,0.857143,5267762,83.318115,833.181150,20.350198
41,青森県,1122,2661,31,2021-07-29,25,0,55.0,2.125000,NaN,5.250000,1275783,20.857779,208.577791,4.311078
19,岩手県,1683,1962,47,2021-07-29,12,0,83.0,-0.294118,0.192066,1.400000,1235517,15.879992,158.799919,6.717836
11,宮城県,2244,9791,91,2021-07-29,36,0,165.0,-0.181818,0.669752,1.000000,2292385,42.710976,427.109757,7.197744
36,秋田県,2805,1006,20,2021-07-29,5,0,20.0,0.250000,0.464286,1.500000,985416,10.208886,102.088864,2.029600
15,山形県,3366,2119,47,2021-07-29,18,0,28.0,5.000000,NaN,3.500000,1082296,19.578747,195.787474,2.587093
35,福島県,3927,5587,162,2021-07-29,88,0,345.0,0.100000,0.495174,5.285714,1881981,29.686803,296.868034,18.331747
38,茨城県,4488,12067,167,2021-07-29,166,0,703.0,-0.144330,0.176057,0.930233,2921436,41.305029,413.050294,24.063508
27,栃木県,5049,8034,84,2021-07-29,142,0,519.0,0.392157,0.283016,3.303030,1965516,40.874763,408.747627,26.405280
37,群馬県,5610,8550,155,2021-07-29,103,0,334.0,0.471429,0.362829,5.437500,1969439,43.413378,434.133781,16.959144


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      19142.234043
std       36635.148334
min         699.000000
25%        2680.000000
50%        5587.000000
75%       12044.500000
max      210610.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,7293,210610,2288,2021-07-29,3865,3,15569.0,0.216556,0.173615,0.953007,13834925,152.230677,1522.306771,112.534040
1,大阪府,15147,112213,2721,2021-07-29,932,2,3978.0,0.167920,0.179046,1.021692,8849635,126.799580,1267.995799,44.951006
2,神奈川県,7854,80350,985,2021-07-29,1164,2,5242.0,0.107517,0.108580,0.847619,9209442,87.247414,872.474141,56.919844
3,埼玉県,6171,55051,848,2021-07-29,864,1,3970.0,-0.005754,0.104066,0.694118,7390054,74.493366,744.933664,53.720852
4,愛知県,12903,53578,1008,2021-07-29,250,1,1006.0,-0.056604,0.235231,0.712329,7575530,70.725085,707.250846,13.279599
5,千葉県,6732,47618,732,2021-07-29,576,1,2981.0,-0.001733,0.120697,0.679300,6319772,75.347655,753.476549,47.169423
6,北海道,561,43890,1424,2021-07-29,260,0,1072.0,0.145374,0.146511,0.857143,5267762,83.318115,833.181150,20.350198
7,兵庫県,15708,43394,1316,2021-07-29,280,1,1139.0,0.102362,0.372346,0.879195,5549568,78.193474,781.934738,20.524120
8,福岡県,22440,38388,532,2021-07-29,366,0,1592.0,-0.096296,0.206108,1.652174,5129841,74.832729,748.327287,31.034100
9,沖縄県,26367,23940,235,2021-07-29,392,2,1617.0,0.129683,0.353052,1.562092,1481547,161.587854,1615.878538,109.142673


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      45.269532
std       34.060695
min       10.208886
25%       23.738525
50%       31.392538
75%       55.935737
max      161.587854
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
26,東京都,7293,210610,2288,2021-07-29,3865,3,15569.0,0.216556,0.173615,0.953007,13834925,152.230677,1522.306771,112.534040
28,沖縄県,26367,23940,235,2021-07-29,392,2,1617.0,0.129683,0.353052,1.562092,1481547,161.587854,1615.878538,109.142673
32,神奈川県,7854,80350,985,2021-07-29,1164,2,5242.0,0.107517,0.108580,0.847619,9209442,87.247414,872.474141,56.919844
7,埼玉県,6171,55051,848,2021-07-29,864,1,3970.0,-0.005754,0.104066,0.694118,7390054,74.493366,744.933664,53.720852
5,千葉県,6732,47618,732,2021-07-29,576,1,2981.0,-0.001733,0.120697,0.679300,6319772,75.347655,753.476549,47.169423
31,石川県,9537,4905,117,2021-07-29,94,1,517.0,-0.210084,0.142583,0.516129,1139612,43.040965,430.409648,45.366318
9,大阪府,15147,112213,2721,2021-07-29,932,2,3978.0,0.167920,0.179046,1.021692,8849635,126.799580,1267.995799,44.951006
34,福岡県,22440,38388,532,2021-07-29,366,0,1592.0,-0.096296,0.206108,1.652174,5129841,74.832729,748.327287,31.034100
1,京都府,14586,17966,249,2021-07-29,164,0,678.0,-0.062857,0.244878,1.309859,2545899,70.568393,705.683925,26.631064
27,栃木県,5049,8034,84,2021-07-29,142,0,519.0,0.392157,0.283016,3.303030,1965516,40.874763,408.747627,26.405280


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    44.000000
mean      0.327763
std       0.272560
min      -0.064116
25%       0.166911
50%       0.240054
75%       0.390762
max       1.531277
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
39,長崎県,23562,3405,69,2021-07-29,21,0,97.0,-0.125000,1.531277,1.333333,1350769,25.207863,252.078631,7.181095
12,宮崎県,25245,3199,27,2021-07-29,16,0,62.0,0.600000,0.809890,2.200000,1095903,29.190540,291.905397,5.657435
13,富山県,8976,2223,38,2021-07-29,24,0,100.0,0.142857,0.722789,1.000000,1055999,21.051156,210.511563,9.469706
43,香川県,20757,2214,32,2021-07-29,33,0,67.0,2.000000,0.717857,7.250000,981280,22.562368,225.623675,6.827817
11,宮城県,2244,9791,91,2021-07-29,36,0,165.0,-0.181818,0.669752,1.000000,2292385,42.710976,427.109757,7.197744
8,大分県,24684,3614,64,2021-07-29,20,0,69.0,0.666667,0.662771,9.000000,1151229,31.392538,313.925379,5.993595
33,福井県,10098,1547,36,2021-07-29,27,0,113.0,0.588235,0.609736,2.375000,780053,19.831986,198.319858,14.486195
2,佐賀県,23001,2699,24,2021-07-29,25,0,70.0,0.315789,0.536609,2.125000,823810,32.762409,327.624088,8.497105
35,福島県,3927,5587,162,2021-07-29,88,0,345.0,0.100000,0.495174,5.285714,1881981,29.686803,296.868034,18.331747
36,秋田県,2805,1006,20,2021-07-29,5,0,20.0,0.250000,0.464286,1.500000,985416,10.208886,102.088864,2.029600


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
20,島根県,17952,699,2,2021-07-29,9,0,54.0,0.285714,-0.064116,-0.550000,679324,10.289641,102.896409,7.949079
42,静岡県,12342,10596,155,2021-07-29,108,0,568.0,-0.100000,0.092627,0.384615,3708556,28.571768,285.717676,15.315934
7,埼玉県,6171,55051,848,2021-07-29,864,1,3970.0,-0.005754,0.104066,0.694118,7390054,74.493366,744.933664,53.720852
32,神奈川県,7854,80350,985,2021-07-29,1164,2,5242.0,0.107517,0.108580,0.847619,9209442,87.247414,872.474141,56.919844
5,千葉県,6732,47618,732,2021-07-29,576,1,2981.0,-0.001733,0.120697,0.679300,6319772,75.347655,753.476549,47.169423
0,三重県,13464,5685,114,2021-07-29,34,0,146.0,-0.081081,0.122182,0.259259,1813859,31.342017,313.420172,8.049137
18,岡山県,18513,8043,127,2021-07-29,59,0,261.0,0.017241,0.124939,0.787879,1903627,42.250924,422.509242,13.710669
31,石川県,9537,4905,117,2021-07-29,94,1,517.0,-0.210084,0.142583,0.516129,1139612,43.040965,430.409648,45.366318
4,北海道,561,43890,1424,2021-07-29,260,0,1072.0,0.145374,0.146511,0.857143,5267762,83.318115,833.181150,20.350198
29,滋賀県,14025,5918,93,2021-07-29,32,0,175.0,-0.360000,0.155603,0.454545,1420948,41.648252,416.482517,12.315722


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   699.     26937.875  53176.75   79415.625 105654.5   131893.375
 158132.25  184371.125 210610.   ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([37.,  2.,  5.,  1.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 7., 19.,  9.,  7.,  2.,  0.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
19625,2021-07-20,35,山口県,3,3196,0,80,19.0,inf,NaN,0.500000
19626,2021-07-21,35,山口県,4,3200,0,80,20.0,0.333333,NaN,0.333333
19627,2021-07-22,35,山口県,4,3204,0,80,23.0,0.000000,NaN,3.000000
19628,2021-07-23,35,山口県,0,3204,0,80,19.0,-1.000000,NaN,-1.000000
19629,2021-07-24,35,山口県,2,3206,0,80,15.0,inf,NaN,-0.666667
19630,2021-07-25,35,山口県,5,3211,0,80,18.0,1.500000,NaN,1.500000
19631,2021-07-26,35,山口県,7,3218,0,80,25.0,0.400000,NaN,inf
19632,2021-07-27,35,山口県,9,3227,0,80,31.0,0.285714,NaN,2.000000
19633,2021-07-28,35,山口県,10,3237,0,80,37.0,0.111111,NaN,1.500000
19634,2021-07-29,35,山口県,16,3253,0,80,49.0,0.600000,NaN,3.000000
